In [1]:
import ee
import requests
import os
# Initialize the Earth Engine API
ee.Initialize(project='ts-rishimnb')


In [2]:
import rasterio
import numpy as np
def get_data(start_date, end_date):
    """
    Downloads the NDVI band from the MODIS image collection as a GeoTIFF file for the given date range.

    Parameters:
    start_date (str): The start date in the format 'YYYY-MM-DD'.
    end_date (str): The end date in the format 'YYYY-MM-DD'.
    """
    # Define the area of interest (AOI) using coordinates (hardcoded)
    bbox = ee.Geometry.Rectangle([78.00405826, 16.93264351, 79.04971836, 17.90150706])
    
    # Choose the MODIS image collection and filter by date and bounds
    image_collection = ee.ImageCollection("MODIS/061/MOD11A1") \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date)
    
    # Calculate the mean image over the date range
    img = image_collection.mean()
    
    # Generate the download URL for the NDVI band
    url = img.getDownloadUrl({
        'bands': ['LST_Night_1km'],
        'region': bbox,
        'scale': 1000,
        'format': 'GEO_TIFF'
    })
    
    # Download the file
    response = requests.get(url)
    with open('nlst.tif', 'wb') as fd:
        fd.write(response.content)
    
    geo_tiff_path = 'nlst.tif'
    with rasterio.open(geo_tiff_path) as src:
    # Read the first band (MODIS data can have multiple bands)
        band1 = src.read(1)
    return band1

In [7]:
print(get_data('2023-06-01', '2023-06-30'))

[[14998.42857143 15002.57142857 14995.14285714 ... 14950.55555556
  14955.33333333 14958.22222222]
 [14997.71428571 14997.42857143 14993.28571429 ... 14944.77777778
  14951.55555556 14948.        ]
 [14999.71428571 15003.14285714 15003.         ... 14940.66666667
  14948.22222222 14942.11111111]
 ...
 [14921.83333333 14931.16666667 14943.83333333 ... 14930.14285714
  14924.5        14928.2       ]
 [14913.83333333 14914.83333333 14909.83333333 ... 14909.28571429
  14910.16666667 14911.33333333]
 [14898.         14885.         14884.83333333 ... 14911.85714286
  14906.85714286 14905.33333333]]


In [8]:
months = {
    1: ("January", 31),
    2: ("February", 28),
    3: ("March", 31),
    4: ("April", 30),
    5: ("May", 31),
    6: ("June", 30),
    7: ("July", 31),
    8: ("August", 31),
    9: ("September", 30),
    10: ("October", 31),
    11: ("November", 30),
    12: ("December", 31)
}

for year in range(2018,2024):
    for i in range(1,13):
        output_dir=fr"C:\Users\rishi\ml_projects\climate\{year}_data\nlst"
        month_name, end_day = months[i]
        start=f"{year}-{i}-1"
        end = f"{year}-{i}-{end_day}"
        arr=get_data(start,end)
        path=os.path.join(output_dir, f'{month_name}_NLST.csv')
        np.savetxt(path, arr, delimiter=',')

In [10]:
import pandas as pd

df= pd.read_csv(r"C:\Users\rishi\ml_projects\climate\2018_data\nlst\January_NLST.csv")
df

,1.442388888888888869e+04,1.441714814814814781e+04,1.441777777777777737e+04,1.442570370370370438e+04,1.444338461538461524e+04,1.446695999999999913e+04,1.448331999999999971e+04,1.450800000000000000e+04,1.454079999999999927e+04,1.455050000000000000e+04,...,1.448188888888888869e+04,1.447796296296296350e+04.1,1.447607407407407482e+04,1.447751851851851825e+04,1.447251851851851825e+04,1.445496296296296350e+04,1.444414814814814781e+04,1.444555555555555475e+04,1.444229629629629562e+04,1.444811111111111131e+04
0,14426.037037,14427.740741,14428.925926,14435.769231,14450.423077,14474.080000,14492.280000,14523.840000,14548.760000,14544.640000,...,14512.370370,14483.296296,14471.888889,14466.740741,14463.851852,14446.259259,14439.259259,14457.851852,14461.555556,14461.884615
1,14436.185185,14440.666667,14442.481481,14441.777778,14450.111111,14475.192308,14504.600000,14523.200000,14543.640000,14540.920000,...,14490.518519,14469.185185,14458.703704,14458.777778,14454.000000,14441.888889,14440.148148,14457.222222,14464.740741,14469.333333
2,14439.555556,14438.592593,14443.185185,14463.962963,14482.230769,14512.000000,14532.320000,14541.440000,14543.920000,14537.240000,...,14507.407407,14485.222222,14475.185185,14478.740741,14477.296296,14463.000000,14455.518519,14465.370370,14474.925926,14482.370370
3,14446.111111,14442.555556,14454.481481,14471.222222,14490.555556,14524.153846,14552.360000,14556.000000,14555.400000,14558.000000,...,14502.148148,14499.777778,14503.407407,14499.481481,14487.407407,14485.592593,14497.222222,14503.518519,14501.222222,14504.222222
4,14447.333333,14442.481481,14456.000000,14470.333333,14488.814815,14539.153846,14593.666667,14562.760000,14548.040000,14550.640000,...,14488.925926,14488.148148,14485.962963,14485.185185,14483.888889,14489.296296,14505.148148,14514.666667,14510.037037,14504.925926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,14489.692308,14493.000000,14487.461538,14500.040000,14503.160000,14469.888889,14444.074074,14425.037037,14423.259259,14428.592593,...,14555.200000,14545.320000,14524.920000,14514.360000,14509.920000,14515.800000,14526.600000,14543.461538,14549.703704,14550.370370
104,14491.192308,14485.222222,14479.370370,14485.222222,14501.148148,14484.814815,14459.666667,14441.518519,14433.333333,14433.074074,...,14554.560000,14538.920000,14526.520000,14525.280000,14527.800000,14533.560000,14542.320000,14549.923077,14556.846154,14556.888889
105,14480.230769,14474.346154,14471.346154,14471.407407,14470.740741,14467.296296,14465.259259,14455.037037,14446.629630,14445.444444,...,14557.240000,14554.760000,14544.560000,14544.320000,14544.000000,14552.400000,14554.720000,14556.000000,14555.153846,14555.333333
106,14478.692308,14473.423077,14465.884615,14457.629630,14455.888889,14455.777778,14454.259259,14448.814815,14443.925926,14444.000000,...,14537.200000,14552.800000,14552.760000,14544.520000,14549.880000,14558.800000,14555.520000,14551.730769,14553.153846,14557.296296
